In [1]:
# Importação de bibliotecas
import pandas as pd
import numpy as np

In [2]:
# Variáveis utilizadas
fonte = 'sidra'
dataset_name = 'risco_erosao_lavouras_perenes'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

In [3]:
# Tabela de área destinada à colheita das lavouras perenes
df = (
    pd.read_parquet(f'{raiz}/prata_sidra_tab1613_lavoura_permanente_area_destinada_colheita.parquet')
    .query('data >= 2000')
    .reset_index()
    .assign(geocodigo_uf = lambda df: df.geocodigo // 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'area':'sum'})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

### Ouro

In [4]:
df_cafe = (
    df
    .reset_index()
    .query('cultura == 2723')
    .replace({2723: 'Café - Convencional + cobertura do solo'})
    .assign(valor = lambda n: round(n.area / 10000, 3))
    .drop(columns='area')
    .assign(
        risco_erosivo = lambda df: np.where(df.cultura == 'Café - Convencional + cobertura do solo', 2.2,
                                        pd.NA)
    )
    .convert_dtypes()
    .reset_index(drop=True)
)

In [5]:
df_frutas = (
    df
    .reset_index()
    .query('cultura != 2723')
    .groupby(['geocodigo', 'data'])
    .agg({'area':'sum'})
    .reset_index()
    .assign(**{
        'Frutíferas - Sem cobertura do solo (solo nu)': lambda n: round((n.area / 10000) * 0.5, 3),
        'Frutíferas - Com proteção à erosão': lambda n: round((n.area / 10000) * 0.5, 3)
        })
    .drop(columns='area')
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .assign(
        risco_erosivo = lambda df: np.where(df.cultura == 'Frutíferas - Sem cobertura do solo (solo nu)', 0.5,
                                        np.where(df.cultura == 'Frutíferas - Com proteção à erosão', 0,
                                                 pd.NA))
    )
    .convert_dtypes()
)

In [6]:
df_final = (
    pd.concat([
        df_cafe,
        df_frutas
    ])
    .sort_values(['geocodigo', 'data'])
    .reset_index(drop=True)
)

In [7]:
df_ouro = (
    df_final
    .rename(columns={
        'geocodigo': 'state',
        'valor': 'value',
        'data': 'date',
        'cultura': 'label',
        'risco_erosivo': 'sub_label'
        })
    .assign(country = 76)
    .filter(['country', 'state', 'date', 'label', 'sub_label', 'value'])
)

df_ouro.to_csv(f'{raiz}/ouro_{dataset_name}.csv', index=False)